#Яся ду
Здравствуйте! Перед вами небольшая викторина-история по питону, которая поможет подготовиться к экзамену. В викторине присутствует 3 типа вопросв: выбор варианта ответа, текстовый и "да/нет", - за правильный ответ в норме даётся по 1 баллу. На первые два типа вопросов пользователю отводится 2 попытки для того, чтобы ответить правильно, однако, если получилось выбрать правильный ответ только со второго балла, он будет весить 0.5 вместо классического 1. Вопросы "да/нет" весят либо 1, либо 0, однако, у пользователя есть неограниченно еколичество попыток, чтобы корректно ввести одно из этих двух слов. Вопросы с выбором ответа также проверяют корректность ввода.

**Как работает программа?**

1. Считывает 2 файла: в первом ххранятся вопросы, варианты ответов и комментарии к ответам, во втором - текстовая часть истории.
2. Соритрует вопросы и ответы по словарям
3. Предлагает пользователю пройти викторину
4. В случае, если пользователь согласен, выводит небольшую предысторию и вопросы. Если вопрос предполагает выбор варианта ответа, то также выводит варианты
5. Считывает ответ пользователя. Если вопрос предполагает проверку на корректность, то просит ввести корректный ответ. 
6. Проверяет ответ. В случае, если вопрос предполагает 2 попытки, то предлагает пользователю подумать езё раз, если первый вариант оказался неправильным.
7. Параллельно суммирует баллы за правильные ответы и записывает информацию о вопросах, правильных ответах и ответах пользователя в файл 'result.txt'.
8. Выводит результат, а также комментарий к нему и конец истории. Результат также записывается в файл.

In [21]:
questions = {}
answers = {}
text = {}

points = {(0, 1, 2, 3, 4): ['- Ну что ж, дорогая Яся, всё же стоит подготовиться получше, - раздосадованно вздохнула преподавательница.', 
                            '- Чёрт! Как хорошо, что есть время ещё подшготовиться!'],
          (5, 6, 7): ['- Неплохо, но можно подготовиться получше, - отметила преподавательница что-то на своём листке.', 
                      '- Отлично, база есть, осталось подтянуть несколько тем.'],
          (8, 9, 10): ['- Замечательно, просто превосходный результат! - обрадованно прокомментировала учительница.',
                       '- Не зря готовилась, - с облегчением выдохнула Яся. - Ну теперь я точно уверена в своих силах.']}
#создаём необходимые словари для вопросов, ответов и текста (так его просто удобнее выводить и группировать, имхо)
#также создаём словарь для распределения ответов в зависимости от баллов игрока, он понадобится нам позже
def sort(lines): #функция, чтобы рассортировать вопросы и ответы в словари. 
                #да, необязательно было это выносить в отдельную функцию, но чисто визуально приятнее работать с отдельным блоком сортировки в самом начале, чем потом искать его в общем коде
  i = 0
  while i < len(lines): #цикл сделан именно через while, так как внутри цикла меняется i, а с for такого не сделать
    if lines[i][0].isdigit(): #в файле есть как вопросы, так и ответы, и комментарии к ним. Проверям, что это вопрос
      number, type_of_question, text_of_question = lines[i].split('.') #в одной строке содержится через точку информация о номере вопроса, его типе и текст. выделяем их
      number = int(number)
      text_of_question = text_of_question.split('/')
      questions[number] = [type_of_question, text_of_question] #добавляем в словарь с вопросами с ключом-номером
      answers[number] = {'comment': [], 'text': [], 'user-1': '', 'user-2': ''} #создаём для вопроса подсловарь и задаём его названия, чтобы потом не думать о них
      if type_of_question == 'D': #если это вопрос с выбором варианта ответа, то он содержит 4 варианта ответа и 4 комментария (3 относительно ответов пользователя и 1 общий)
        for i in range(i+1, i+5):
          answers[number]['text'].append(lines[i].split('.'))
        for i in range (i+1, i+4):
          answers[number]['comment'].append(lines[i])
        i+=1
        questions[number].append(lines[i]) #общий комментарий кидаем к вопросу, так как у каждого типа вопроса он есть, и в дальнейшем это облегчит вывод
      elif type_of_question == 'T': #вопросы с полем для ответа не имеют вариантов, но зато у них 4 комментария (3 относительно ответов пользователя и 1 общий)
        i+=1
        answers[number]['text'] = [lines[i].split('.')]
        for i in range(i+1, i+4):
          answers[number]['comment'].append(lines[i])
        i+=1
        questions[number].append(lines[i])
      elif type_of_question == 'YN': #вопросы "да/нет" не имеют вариантов, зато имеют 3 комментария (2 относительно ответов пользователя и 1 общий)
                                    #у "да-неток" меньше комментариев, так как только эти вопросы не подразумевают "второй попытки" 
                                    #(если овтет "да" неправильный, то правильный ответ "нет", и во второй раз пользователь практически всегда ответит верно)
        i+=1
        answers[number]['text'] = lines[i].split('\n')[0] #ответы записываются с маркером переноса на другую строку, поэтому нам лучше избавиться от него, записав в правильный ответ всё, что до него
        for i in range(i+1, i+3):
          answers[number]['comment'].append(lines[i])
        i+=1
        questions[number].append(lines[i])
      i+=1

def yn(n): #функция для проверки вопросов на "да/нет". выделила в функцию, так как часто встречается в программе и не хочется занимать этим место в основной части
  while n!='да' and n!='нет': #пока ответ не "да/нет": раз за разом предлагаем ввести ответ и возвращаем тот вариант, что прошёл проверку
    n = input('Пожалуйста, введите корректный ответ: ')
  return n

def yd(n): #функция для проверки ответа: является ли он цифрой. также выделила по принципу "часто встречается"
  while not n.isdigit(): #принцип работы схож с предыдущей функцией
    n = input('Пожалуйста, введите корректный ответ: ')
  return n

def n_is_correct(n, counter, res, number): #проверка ответа на правильность и "второй шанс"
                                          #вызывает саму себя, поэтому 100% делаем функцией, а не внутри основного кода
  counter+=1 #номер попытки ответа прибавляется, когда только начинает работу цикл, чтобы потом иметь ввиду, работам мы со 2 попыткой или 1
  if counter == 1: #запись ответа пользователя в словарь в зависимсоти от попытки, а также вывод в отчёт
    answers[number]['user-1'] = n
    res.write(f'Первый ответ пользователя: {n}\n')
  else:
    answers[number]['user-2'] = n
    res.write(f'Второй ответ пользователя: {n}\n')
  for i in range(len(answers[number]['text'])): #у текстовых и вопросов с выбором первым элементом списка стоит бинарная перемена "правильности": ищем из списка номер праильного ответа
    if answers[number]['text'][i][0] == '1':
      res.write(f"Правильный ответ: {i+1}. {answers[number]['text'][i][1]}\n")
      if (questions[number][0] == 'D' and int(n) == (i+1)) or (questions[number][0] == 'T' and n == answers[number]['text'][i][1].split('\n')[0]):
        #извиняюсь за такое условие, но оно надо, так как для проверки правильности вопроса с выбором ответа нужно сравнить порядковый номер ответа в списке с введенным значением, а для текстовых - значение списка с ответом
        print(answers[number]['comment'][2]) #выводим комментарий для правильного ответа
      else: 
        print(answers[number]['comment'][counter-1]) #выводим комментарий в соответствии с номером попытки: подсказка или разочарование
        if counter == 1:
          n = input('Введите новый вариант ответа: ') #предлагаем пользователю попробовать снова, если у него не получилось с первого раза
          if questions[number][0] == 'D': 
            n = yd(n) #не забываем проверять ответы пользователя на цифровость
          n_is_correct(n, counter, res, number)#отправляем ответ пользователя на перепроверку

def print_answer(number, res): #функция для вывода вариантов ответа. зачем? так проще ориентирвоаться в смысловом содержании куска кода
  if questions[number][0] == 'D':
    for i in range(len(answers[number]['text'])):
      print(f"{i+1}. {answers[number]['text'][i][1]}") #выводим номер варианта ответа и сам ответ, если это "цифры"
      res.write(f"{i+1}. {answers[number]['text'][i][1]}")
    n = yd(input('Введите цифру от 1 до 4: '))
    n_is_correct(n, 0, res, number)
  elif questions[number][0] == 'T': #в текстовом вопросе нет вариантов ответа, поэтому мы просто выводим два разных input'а
    if number == 11:
      n = input('Введите ФИ: ') #11 вопрос требует уточнения, что именно нужно ввести6 выделяем его отдельно
    else:
      n = input('Введите текст Вашего ответа: ')
    n_is_correct(n, 0, res, number) #проверяем правильность
  elif questions[number][0] == 'YN':
    n = yn(input('Введите "да" или "нет": '))
    answers[number]['user-1'] = n #сразу записываем в ответ пользователя, так как тут никаикх вторых шансов
    if answers[number]['user-1'] == answers[number]['text']:
      print(answers[number]['comment'][1]) #выводим результаты согласно "правильности ответа пользователя"
    else:
      print(answers[number]['comment'][0])
    res.write(f'Ответ пользователя: {n}')
    res.write(f"Правильный ответ: {answers[number]['text'][0]}")

def print_results(correct): #функция для поиска ключа, в котором лежит реакция на результаты викторины
  for cort in points: #по каждому кортежу смотрим, есть ли там округленный вариант набранных баллов
    if round(correct) in cort:
      return cort #когда будет - возвращаем ключ, то есть, кортеж

with open('Python.txt', mode = 'r', encoding ='utf-8-sig') as python:
  lines = python.readlines()
sort(lines)
with open('Python_text.txt', mode = 'r', encoding = 'utf-8-sig') as textfile:
  lines_of_text = textfile.readlines()
  for line in lines_of_text:
    if line[0].isdigit(): #файл с текстом для вывода разделён таким образом, что цифры символизируют начало текста, в котором может быть несколько строк
      number = line.split('.')[0]
      text_text = line.split('.')[1::]
      text[int(number)] = ['.'.join(text_text)]
    else:
      text[int(number)].append(line)
    #соответственно, если строка начинается с цифры - мы записываем её в словарь с новым ключом, а если нет - добавляем к списку с уже известным ключом
    
with open('result.txt', mode = 'w', encoding = 'utf-8') as res: #открываем файл для записи, так как мы многозадачные
  print(*text[1])
  n = yn(input('\nВведите "да" или "нет":')) #вежливо спрашиваем, хотит ли пользователь сыграть
  if n == 'нет':
    print(*text[2]) #не хочет - у нас есть специальный текст на эту штуку
  else: #если хочет - запускам основные штуки викторины
    correct = 0 #переменная для подсчёта баллов пользователя
    print(*text[3])
    n = yn(input('\nВведите "да" или "нет":')) #вопрос от преподавателя создаёт иллюзию выбора, пользователь уже согласился поучаствовать. прости, пользователь
    print(*text[4])
    for number in questions:
      print(f'Вопрос номер {number}.') #выводим номер вопроса
      res.write(f'Вопрос номер {number}.')
      for line in questions[number][1]: #так как некоторые вопрсы включают в себя переносы строк, то выводим их построчно
        print(line)
        res.write(line)
      print_answer(number, res)
      if questions[number][0] == 'D':
        for i in range(len(answers[number]['text'])):
          if answers[number]['text'][i][0] == '1' and i+1 == int(answers[number]['user-1']): #сравниваем первый ответ пользователя с правильным, который мы нашли по старой методике
            correct+=1 #если всё хорошо ещё на первой попытке, то у пользователя не будет заполнено user-2, можно без зазрений совести давать балл
          elif answers[number]['text'][i][0] == '1' and i+1 == int(answers[number]['user-2']):
            correct+=0.5 #а если попытки 2, то первая точно непраивльная, мы проверяем вторую и либо даём полбалла, либо оставляем ни с чем
      elif questions[number][0] == 'T' and number!=11:
        if answers[number]['user-1'] == answers[number]['text'][0][1].split('\n')[0]:
          correct+=1 #такая же схема подсчёта баллов для текстовых ответов, но!
        elif answers[number]['user-2'] == answers[number]['text'][0][1].split('\n')[0]:
          correct += 0.5
        #почемы мы исключаем 11 вопрос из подсчёта? потому что он существует чисто ради шутки и не влияет на оценку
      elif questions[number][0] == 'YN':
        if answers[number]['user-1'] == answers[number]['text']:
          correct+=1
          #если вариант ответа "да/нет", то мы либо даём балл, либо нет, так как второго ответа нет и быть не может
      print(questions[number][2]) #вывод общего комментария по ответу. 
      #выводить его проще так и добавлять в словарь к вопросу, ведь для разных типов вопросов разное кол-во комментариев, и он бы оказался под разным иномерами или пришлось создавать допключ...
    print(f"Твой результат: {correct} баллов из 10") #сразу выводим уже подсчитанные результаты
    res.write(f"\nРезультат игрока: {correct} баллов из 10")
    print(points[print_results(correct)][0]) #выводим реакцию преподавательнцы
    res.write(points[print_results(correct)][0])
    print(*text[5])
    print(points[print_results(correct)][1]) #выводим самооценку Яси
    print(*text[6])      

Привет! Многих из нас очень волнует вопрос подготовки к экзамену по Питону, и девочка Яся попала в очень неловкую ситуацию. Ей очень нужна твоя помощь, чтобы не потерять лицо перед преподавателем, не подсобишь? Заодно и свои знания проверишь.


Введите "да" или "нет":нет
Понимаю, удачной подготовки.

